Following the [`theano` Tutorial](http://deeplearning.net/software/theano/tutorial/index.html)

In [1]:
from theano import *
import theano.tensor as T

## Baby Steps - Algebra
### Adding two Scalars

In [2]:
import numpy
from theano import function

In [3]:
x = T.dscalar('x')
y = T.dscalar('y')
z = x+y
f = function([x,y],z)

In [7]:
print type(x), type(y), type(z), type(f) # good to know what these 
# new classes are in theano

<class 'theano.tensor.var.TensorVariable'> <class 'theano.tensor.var.TensorVariable'> <class 'theano.tensor.var.TensorVariable'> <class 'theano.compile.function_module.Function'>


In [8]:
f(2,3)

array(5.0)

In [9]:
numpy.allclose(f(16.3,12.1),28.4)

True

In [10]:
x.type

TensorType(float64, scalar)

In [11]:
T.dscalar

TensorType(float64, scalar)

In [12]:
x.type is T.dscalar

True

"Prefer constructors like `matrix, vector` and `scalar` to `dmatrix, dvector` and `dscalar` because the former will give you `float32` variables when `floatX=float32`." - cf. [Using the GPU Theano tutorial](http://deeplearning.net/software/theano/tutorial/using_gpu.html)

In [13]:
xf = T.scalar('xf')
yf = T.scalar('yf')
zf = xf + yf
ff = function([xf,yf],zf)

In [14]:
from theano import pp

In [15]:
print(pp(z))

(x + y)


In [16]:
print(pp(zf))

(xf + yf)


In [17]:
x = T.dmatrix('x')
y = T.dmatrix('y')
z = x + y 
f = function([x,y],z)

In [18]:
f([[1,2],[3,4]], [[10,20],[30,40]])

array([[ 11.,  22.],
       [ 33.,  44.]])

In [19]:
f(numpy.array([[1,2],[3,4]]),numpy.array([[10,20],[30,40]]))

array([[ 11.,  22.],
       [ 33.,  44.]])

In [20]:
xf = T.matrix('xf')
xy = T.matrix('yf')
zf = xf + yf
ff = function([xf,yf],zf)

In [22]:
ff([[1,2],[3,4]], [[10,20],[30,40]])

TypeError: ('Bad input argument to theano function with name "<ipython-input-20-f711db7fb599>:4"  at index 1(0-based)', 'Wrong number of dimensions: expected 0, got 2 with shape (2, 2).')

Adding exercise 1, cf. http://deeplearning.net/software/theano/tutorial/adding.html

In [25]:
a = theano.tensor.vector()
b = theano.tensor.vector()
out = a**2 + b**2 + 2 * a * b
f = theano.function([a,b],out)
print(f([1,2],[4,5]))

[ 25.  49.]
